In [ ]:
import cv2
import time
import os
import numpy as np
from PIL import Image
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Set the folder to save and load images
save_folder = r"C:\Users\HP\OneDrive\Desktop\DIP\Hair"
os.makedirs(save_folder, exist_ok=True)

def choose_image_source():
    choice = input("Choose image source (1 for Real-Time capture, 2 for Gallery import): ")
    return choice

def capture_real_time_image():
    new_image_path = os.path.join(save_folder, "new_hair_photo.png")

    cap = cv2.VideoCapture(0)
    start_time = time.time()

    print("Position yourself properly. Auto-capture in 10 seconds...")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not access the camera!")
            break

        # Display live webcam feed
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(frame_rgb)
        clear_output(wait=True)
        display(img_pil)

        if time.time() - start_time >= 10:
            cv2.imwrite(new_image_path, frame)
            print(f"New hair photo saved at:\n{new_image_path}")
            break

    cap.release()
    return new_image_path
    new_image_path = os.path.join(save_folder, new_image_filename)


def import_from_gallery():
    new_image_filename = input("Enter NEW image filename (e.g., new_hair.jpg): ")
    new_image_path = os.path.join(save_folder, new_image_filename)

    if os.path.exists(new_image_path):
        print(f"Image {new_image_filename} successfully imported!")
    else:
        print(f"Error: {new_image_filename} does not exist in the specified folder.")
        return None
    return new_image_path
    new_image_path = os.path.join(save_folder, new_image_filename)


def analyze_hair(old_image_path, new_image_path):
    if not os.path.exists(old_image_path):
        print("Old image not found!")
        return
    if not os.path.exists(new_image_path):
        print("New image not found!")
        return

    # Load images
    old_img = cv2.imread(old_image_path)
    new_img = cv2.imread(new_image_path)

    if old_img is None or new_img is None:
        print("Error reading one or both images.")
        return

    # Resize new image to match old
    new_img = cv2.resize(new_img, (old_img.shape[1], old_img.shape[0]))

    # Convert to grayscale
    gray1 = cv2.cvtColor(old_img, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    canny_edges1 = cv2.Canny(gray1, threshold1=100, threshold2=200)
    canny_edges2 = cv2.Canny(gray2, threshold1=100, threshold2=200)

    # Crop center area (focus on hair region)
    h1, w1 = canny_edges1.shape
    h2, w2 = canny_edges2.shape

    canny_crop1 = canny_edges1[:, int(w1 * 0.2):int(w1 * 0.8)]
    canny_crop2 = canny_edges2[:, int(w2 * 0.2):int(w2 * 0.8)]

    # Calculate edge density
    canny_density1 = np.count_nonzero(canny_crop1) / canny_crop1.size
    canny_density2 = np.count_nonzero(canny_crop2) / canny_crop2.size

    print("\n--- Hair Edge Density Comparison ---")
    print("Old Image Edge Density:", round(canny_density1, 4))
    print("New Image Edge Density:", round(canny_density2, 4))
    
  # Calculate percentage change in density
    if canny_density1 != 0:
        percentage_change = ((canny_density1 - canny_density2) / canny_density1) * 100
    else:
        percentage_change = 0

    # Classify the result based on the percentage change
    if abs(percentage_change) <= 10:
        classification = "Unchanged"
    elif percentage_change > 10:
        classification = "Hair Thickening"
    else:
        classification = "Hair Thinning"
    print(f"Percentage Change: {round(percentage_change, 2)}%")
    print(f"Classification: {classification}")
        
    # Show images and their edge maps
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))

    axs[0, 0].imshow(cv2.cvtColor(old_img, cv2.COLOR_BGR2RGB))
    axs[0, 0].set_title("Original Old Image")
    axs[0, 0].axis('off')

    axs[0, 1].imshow(canny_edges1, cmap='gray')
    axs[0, 1].set_title("Canny Edges - Old")
    axs[0, 1].axis('off')

    axs[1, 0].imshow(cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB))
    axs[1, 0].set_title("Original New Image")
    axs[1, 0].axis('off')

    axs[1, 1].imshow(canny_edges2, cmap='gray')
    axs[1, 1].set_title("Canny Edges - New")
    axs[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

    # Save a backup copy of old image
    saved_old_path = os.path.join(save_folder, "saved_old_hair.png")
    cv2.imwrite(saved_old_path, old_img)
    print(f"\nOld image backed up to: {saved_old_path}")

def main():
    choice = choose_image_source()

    if choice == '1':
        new_image_path = capture_real_time_image()
    elif choice == '2':
        new_image_path = import_from_gallery()
        if new_image_path is None:
            return
    else:
        print("Invalid choice! Please enter 1 or 2.")
        return

    # Ask for old image to compare
    old_image_filename = input("Enter OLD image filename (e.g., old_hair.jpg): ")
    old_image_path = os.path.join(save_folder, old_image_filename)
    # Run analysis
    analyze_hair(old_image_path, new_image_path)

if __name__ == "__main__":
    main()

Choose image source (1 for Real-Time capture, 2 for Gallery import):  2
